In [103]:
import numpy as np

In [129]:
X = np.random.normal(0, 1, size=(1000, 20))
y = np.random.randint(4, size=1000)

In [131]:
class Layer(object):
    
    def __init__(self, input_size, output_size, activation):
        
        self.input_size = input_size
        self.output_size = output_size
        
        activations = {
            "none": lambda z: z,
            "relu": lambda z: max(0, z),
            "sigmoid": lambda z: 1/(1 + np.exp(-z)),
            "softmax": lambda z: np.exp(z)/np.exp(z).sum()
        }
        
        self.activation_name = activation
        self.activation_fun = activations.get(activation)
        
    
    def update(self, grads):
        pass
    
    

class Input(Layer):
    
    def __init__(self, output_size, activation="none"):
        super().__init__(output_size, activation)

In [137]:
class Dense(Layer):
    
    def __init__(self, input_size, output_size, activation="sigmoid"):
        super().__init__(input_size, output_size, activation)
        self.weights = np.random.uniform(-0.1, 0.1, size=(self.input_size, self.output_size))

    def process(self, inputs):
        raw_output = np.matmul(inputs, self.weights)
        return self.activation_fun(raw_output)
    
    
    
    
    
    def update_weights(self, meh):
        pass

In [138]:
X = np.random.normal(0, 1, size=(1000, 20))

In [139]:
layer1 = Dense(20, 10, "sigmoid")

In [187]:
class Network(object):
    
    def __init__(self, layers):
        self.layers = layers
        self.output_size = layers[-1].output_size
    
    
    def show(self):
        k = 1
        for layer in self.layers:
            print("Layer {0}: {1}, {2} -> {3}, {4}".format(
                k, 
                type(layer).__name__,
                layer.input_size,
                layer.output_size,
                layer.activation_name
            ))
            k = k+1
    
    
    def predict(self, data, classes=True):
        outputs = data
        for k in range(len(self.layers)):
            outputs = self.layers[k].process(outputs)
        return np.argmax(outputs, axis=1) if classes else outputs
    
    
    def get_cost(self, data, labels, penalty=0):
        m = data.shape[0]
        preds = self.predict(data, classes=False)
        
        cost = (-1/m) * (np.log(preds) * np.eye(self.output_size)[:, labels].T +
                         np.log(1-preds) * (1 - np.eye(self.output_size)[:, labels]).T).sum()
        
        # Add regularisation
        cost = cost + (penalty/m) * sum([(layer.weights**2).sum() for layer in self.layers])
        
        return cost

    

In [188]:
model = Network(
    layers=[
        Dense(20, 10, "sigmoid"),
        Dense(10, 4, "softmax")
    ]
)

In [189]:
model.show()

Layer 1: Dense, 20 -> 10, sigmoid
Layer 2: Dense, 10 -> 4, softmax


In [190]:
model.predict(X, classes=False).shape

(1000, 4)

In [192]:
model.get_cost(X, y, penalty=1)

8.298208736480415